In [1]:
# Random Forest Algorithm
# seeding the generated number makes our results reproducible (good for debugging)
# This module implements pseudo-random number generators for various distributions.
from csv import reader
from datetime import datetime
from math import sqrt
from matplotlib import pyplot
from random import randrange
from random import seed
from sklearn import metrics

from sklearn import preprocessing
from sklearn import svm, datasets
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,log_loss
from sklearn.metrics import auc
from sklearn.metrics import classification_report,confusion_matrix,roc_curve,roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import itertools
import matplotlib.pyplot as plt
import numpy as np
import numpy as npX
import pandas as pd
import seaborn as sns

import time

#different models to compare:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC

In [17]:
tweets = pd.read_csv("train.csv")
tweets.head()
tweets.drop
#tweets.fillna(0)
#np.isnan(tweets.values.any())
#df = tweets






,tweet_id,priority,category,event_id,identifier,description,id,desc,level,intentType,exampleLowLevelTypes,eventID,eventName,eventDescription,eventType,postID,postCategories,postPriority
0,211565974422425600,0.75,ServiceAvailable,19.0,TRECIS-ITR-H-Types-v4-T1,TREC-IS Incident Tweet Routing (High-Level) In...,Report-OriginalEvent,A report of the original event occuring. This ...,High-level,Report,[],fireColorado2012,DEMO: CrisisLex26 2012 Colorado wildfires,The 2012 Colorado wildfires were an unusually ...,Unknown,211565974422425600,['ServiceAvailable'],High
1,211654415503990784,0.50,News,15.0,TRECIS-ITR-H-Types-v4-T1,TREC-IS Incident Tweet Routing (High-Level) In...,Report-Official,An official report by a government or public s...,High-level,Report,"['OfficialStatement', 'RegionalWarning', 'Publ...",fireColorado2012,DEMO: CrisisLex26 2012 Colorado wildfires,The 2012 Colorado wildfires were an unusually ...,Unknown,211654415503990784,['News'],Medium
2,211681309368655872,0.25,News,15.0,TRECIS-ITR-H-Types-v4-T1,TREC-IS Incident Tweet Routing (High-Level) In...,Report-Official,An official report by a government or public s...,High-level,Report,"['OfficialStatement', 'RegionalWarning', 'Publ...",fireColorado2012,DEMO: CrisisLex26 2012 Colorado wildfires,The 2012 Colorado wildfires were an unusually ...,Unknown,211681309368655872,['News'],Low
3,211685621125742592,0.25,Official,16.0,TRECIS-ITR-H-Types-v4-T1,TREC-IS Incident Tweet Routing (High-Level) In...,Report-News,The post is a news report providing/linking to...,High-level,Other,"['NewsHeadline', 'NewsArticle']",fireColorado2012,DEMO: CrisisLex26 2012 Colorado wildfires,The 2012 Colorado wildfires were an unusually ...,Unknown,211685621125742592,['Official'],Low
4,211877049147736064,0.25,Factoid,7.0,TRECIS-ITR-H-Types-v4-T1,TREC-IS Incident Tweet Routing (High-Level) In...,Report-ThirdPartyObservation,The user is reporting a information that they ...,High-level,Report,"['Group/IndividualMovement', 'PeopleEvacuating...",fireColorado2012,DEMO: CrisisLex26 2012 Colorado wildfires,The 2012 Colorado wildfires were an unusually ...,Unknown,211877049147736064,['Factoid'],Low


In [21]:
labels = tweets.iloc[:, 10].values
features = tweets.iloc[:, 1].values


features[~np.isfinite(features)] = 0



features[features==np.inf]=np.nan
#features.fillna(features.mean(), inplace=True)

#np.any(np.isnan(features))

features

np.all(np.isfinite(features))

True

In [4]:
labels

array(['[]',
       "['OfficialStatement', 'RegionalWarning', 'PublicAlert', 'PressRelease']",
       "['OfficialStatement', 'RegionalWarning', 'PublicAlert', 'PressRelease']",
       "['NewsHeadline', 'NewsArticle']",
       "['Group/IndividualMovement', 'PeopleEvacuating', 'DamageObserved', 'CollapsedStructure', 'ExplosionDamage', 'RoadBlocked', 'CompromisedBridge', 'CommunicationLinesDown']",
       "['Current', 'Forecast']",
       "['Group/IndividualMovement', 'PeopleEvacuating', 'DamageObserved', 'CollapsedStructure', 'ExplosionDamage', 'RoadBlocked', 'CompromisedBridge', 'CommunicationLinesDown']",
       "['LandDevastated', 'InjuriesCount', 'KilledCount', 'NumberDisplacedPeople', 'NumberOfAssailants', 'ShooterCount', 'DamageCost']",
       '[]', "['Video', 'Images', 'Map']",
       "['HospitalOperating', 'ShelterOffered', 'WaterDistributionPoint', 'FoodDistributionPoint']",
       "['HospitalOperating', 'ShelterOffered', 'WaterDistributionPoint', 'FoodDistributionPoint']",
    

In [5]:
import re

features_pp = []
for i in range(0, len(features)):
    feature_pp = re.sub(r'\W', ' ', str(features[i]))
    feature_pp = re.sub(r'\s+[a-zA-Z]\s+', ' ', feature_pp)
    feature_pp = re.sub(r'\^[a-zA-Z]\s+', ' ', feature_pp)
    feature_pp = re.sub(r'\s+', ' ', feature_pp, flags=re.I)
    feature_pp = re.sub(r'^b\s+', '', feature_pp)
    feature_pp = feature_pp.lower()
    features_pp.append(feature_pp)

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

vectorizer = TfidfVectorizer(max_features=2500, 
    min_df=7, max_df=0.8, stop_words=stopwords.words('english'))
features_pp = vectorizer.fit_transform(features_pp).toarray()


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pseudo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    features_pp, labels, test_size=0.2, random_state=0)


In [8]:
#X_test.fillna(X_test.mean())


In [23]:
from sklearn.ensemble import RandomForestClassifier

classifier_rf = RandomForestClassifier(n_estimators=20, random_state=0)
classifier_rf.fit(X_train, y_train)

ValueError: Input contains NaN